#  REGEX - Extracting, cleaning and sorting dates from messy data

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants in the dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once the dates are extracted from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

The function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3
*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import re
import numpy as np

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

print("Sample raw data:")
print(df.iloc[100:140])

Sample raw data:
100                        8/16/92 SOS-10 Total Score:\n
101      10/05/97 CPT Code: 90791: No medical services\n
102    07/18/2002 CPT Code: 90792: With medical servi...
103          9/22/82 Total time of visit (in minutes):\n
104                        2/24/74 SOS-10 Total Score:\n
105    (2/03/78) TSH-0.90 Activities of Daily Living ...
106     2/11/2006 CPT Code: 90791: No medical services\n
107    Pt is a 21 year old, single, heterosexual iden...
108    PET Scan (DPSH 5/04/74): 1) Marked hypometabol...
109                             7/20/2011 [report_end]\n
110          6/17/95 Total time of visit (in minutes):\n
111                        6/10/72 SOS-10 Total Score:\n
112    nPt denied use to me but endorsed use on 10/16...
113    12/15/92 CPT Code: 90801 - Psychiatric Diagnos...
114                        12/8/97 SOS-10 Total Score:\n
115                       4/05/89 Primary Care Doctor:\n
116                       12/04/87 SOS-10 Total Score:\n
117           

In [2]:
def date_sorter():

    date = []
    for i,d in enumerate(df):
        date.append(re.findall(r'\d{1,2}[ /-]\d{1,2}[ /-]\d{4}',d))
    
        if date[i] == []:
            date[i] = (re.findall(r'\d{1,2}[ /-]\d{1,2}[ /-]\d{2}', d))
    
        # option 2: [Day] Mon(th) year
        if date[i] == []:
            date[i] = (re.findall(r'\d{0,2}[ \-]*(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ \-,]*[ ]*\d{4}', d))
    
        # option 3: Mon(th) Day, Year
        if date[i] == []:
            date[i] = (re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ \-.][ ]*\d{2}[ ,\-]*[ ]*\d{4}', d))
        
        # option 4: Mon(th) Year
        if date[i] == []:
            date[i] = (re.findall(r'\d{1,2}[ /-]\d{4}', d))
        
        if date[i] == []:
            date[i] = (re.findall(r'\d{1,2}[ /-]\d{2}', d))
        
        # option 5: Year
        if date[i] == []:
            date[i] = (re.findall(r'\d{4}', d))
    
    
    date = pd.Series(v[0].strip() for v in date )
    dates = date.replace('Decemeber','December',regex=True).replace('Janaury','January',regex=True)
    dates = pd.to_datetime(dates, errors = 'coerce')
    
    

    return dates, pd.Series(pd.Series(dates.sort_values()).index)
   


In [3]:
dates, idx = date_sorter()
print("Exracted and cleaned dates:")
print(dates[100:140])

Exracted and cleaned dates:
100   1992-08-16
101   1997-10-05
102   2002-07-18
103   1982-09-22
104   1974-02-24
105   1978-02-03
106   2006-02-11
107   1983-08-22
108   1974-05-04
109   2011-07-20
110   1995-06-17
111   1972-06-10
112   1982-10-16
113   1992-12-15
114   1997-12-08
115   1989-04-05
116   1987-12-04
117   1977-06-20
118   2006-04-27
119   1992-07-17
120   1998-12-22
121   1996-10-02
122   1990-11-05
123   1977-05-04
124   1996-02-27
125   2001-01-24
126   2004-09-10
127   1982-05-26
128   2002-06-28
129   1972-05-06
130   1987-10-25
131   1996-10-14
132   2007-11-30
133   1994-06-28
134   1981-01-14
135   1985-10-10
136   1985-02-11
137   1983-02-10
138   1992-02-05
139   2012-10-21
dtype: datetime64[ns]


In [4]:
print("Sorted index:")
idx

Sorted index:


0        9
1       84
2        2
3       53
4       28
      ... 
495    186
496    161
497    413
498    392
499    490
Length: 500, dtype: int64